In [ ]:
import json 
import requests
import regex as re

from collections import defaultdict
from tqdm.notebook import trange, tqdm

## Issues

In [ ]:
issues = json.load(open('<issue_json_path>', 'r'))
non_prs = [issue for issue in issues if "pull_request" not in issue.keys()]
prs = [issue for issue in issues if "pull_request" in issue.keys()]

In [ ]:
issues_with_commits = defaultdict(lambda: defaultdict(lambda : {'body': None, 'commits': [], 'labels':[]}))
exceptions = []

In [ ]:
headers = {'Authorization': 'token ' + '<token>'} 

In [ ]:
pb = tqdm(range(len(non_prs)))
for issue in non_prs:
  pb.update(1)
  if issue['timeline_url'] and issue['state_reason']!='not_planned':
    number = issue['number']
    labels = []
    if('labels' in issue.keys()):
      labels = [t['name'] for t in issue['labels']]
    try:
      timeline = requests.get(issue['timeline_url'], headers=headers).json()
    except Exception:
      exceptions.append(number)
      print(number)
      continue 
    has_commit = False 
    commits = []
    for item in timeline:
      if('commit_url' in item.keys() and item['commit_url']!=None):
        has_commit = True 
        commit = requests.get(item['commit_url'], headers=headers).json()
        if('files' in commit.keys()):
          files = [f['filename'] for f in commit['files']]
          commits.append([commit['node_id'], commit['commit']['message'], commit['commit']['committer']['date'], files])
    if(has_commit):
      issues_with_commits[number]['title'] = issue['title']
      issues_with_commits[number]['body'] = issue['body']
      issues_with_commits[number]['commits'] = commits
      issues_with_commits[number]['labels'] = labels
      issues_with_commits[number]['created_at'] = issue['created_at']
      issues_with_commits[number]['closed_at'] = issue['closed_at']

  0%|          | 0/1188 [00:00<?, ?it/s]

In [ ]:
to_save = dict(issues_with_commits)
with open(f'issues/issues.jsonl', 'w') as f:
  for issue in to_save:
    json_record = json.dumps({'issue_number': issue, **to_save[issue]})
    f.write(json_record+'\n')

# PRs

In [ ]:
all_map = {}
for issue in issues:
    number = issue["number"]
    body = issue["body"]
    title = issue["title"]
    created = issue['created_at']
    closed = issue['closed_at']

    all_map[number] = [title, body, created, closed]

In [ ]:
prs_with_commits = defaultdict(lambda: defaultdict(lambda : {'body': None, 'commits': [], 'labels':[]}))
exceptions = []

In [ ]:
pb = tqdm(range(len(prs)))
for pr in prs:
  number = pr['number']
  pb.update(1)
  if(pr['title'][:3]=='DOC'):
    continue
  if(pr['pull_request']['merged_at']!=None):
    if(not(pr["body"])):
      continue
    linked_issues = re.findall(r'(?:fixes|closes) #(\d+)', pr['body'].lower())
    if(len(linked_issues)==0):
      continue
    created_ats = []
    closed_ats = []
    numbers = []
    for issue in linked_issues:
      issue = int(issue)
      created_at = all_map[issue][2]
      closed_at = all_map[issue][3]
      numbers.append(int(issue))
      created_ats.append(created_at)
      closed_ats.append(closed_at)
      labels = []
      if('labels' in pr.keys()):
        labels = [t['name'] for t in pr['labels']]

      try:
        timeline = requests.get(pr['timeline_url'], headers=headers).json()
      except Exception:
        exceptions.append(number)
        print(number)
        continue 

      has_commit = False 
      commits = []
      for item in timeline:
        if('commit_url' in item.keys() and item['commit_url']!=None):
          has_commit = True 
          commit = requests.get(item['commit_url'], headers=headers).json()
          if('files' in commit.keys()):
            files = [f['filename'] for f in commit['files']]
            commits.append([commit['node_id'], commit['commit']['message'], commit['commit']['committer']['date'], files])
            
      if(has_commit):
        prs_with_commits[number]['pr_title'] = pr['title']
        prs_with_commits[number]['pr_body'] = pr['body']
        prs_with_commits[number]['commits'] = commits
        prs_with_commits[number]['labels'] = labels
        prs_with_commits[number]['linked_created_at'] = created_ats
        prs_with_commits[number]['linked_closed_at'] = closed_ats
        prs_with_commits[number]['linked_number'] = numbers

In [ ]:
to_save = dict(prs_with_commits)
with open(f'prs/prs.jsonl', 'w') as f:
  for issue in to_save:
    json_record = json.dumps({'issue_number': issue, **to_save[issue]})
    f.write(json_record+'\n')